In [1]:
from sqlalchemy import Column, String, Integer, ForeignKey, create_engine, func
from sqlalchemy.orm import sessionmaker, declarative_base, relationship

### Engine creation and connection establishment

In [2]:
DATABASE_URL = "sqlite:///CRUDOperations.db"
engine = create_engine(DATABASE_URL, echo=True)
# conn = engine.connect()

### Table Creation

In [3]:
Base = declarative_base()

class User(Base):
    __tablename__ = "user"
    user_id: int = Column(Integer, primary_key=True)
    name: str = Column(String, nullable=False)
    email: str = Column(String, nullable=False)

    blogs = relationship("Blog", back_populates="author")

class Blog(Base):
    __tablename__ = "blog"
    blog_id: int = Column(Integer, primary_key=True)
    title: str = Column(String, nullable=False)
    description: str = Column(String, nullable=False)
    author_id: int = Column(Integer, ForeignKey("user.user_id"))

    author = relationship("User", back_populates="blogs")

In [4]:
Base.metadata.create_all(engine)

2025-09-19 17:39:02,412 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-19 17:39:02,414 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user")
2025-09-19 17:39:02,415 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-19 17:39:02,417 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("blog")
2025-09-19 17:39:02,418 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-19 17:39:02,422 INFO sqlalchemy.engine.Engine COMMIT


### Session creation and accessing the data

In [5]:
Session = sessionmaker(bind=engine)
session = Session()

In [6]:
new_user = User(name="Rashwanth", email="rashwanth@gmail.com")
session.add(new_user)
session.commit()

### Selecting data

In [22]:
kavin = session.query(User).filter(User.name=="Kavin").first()

2025-09-19 17:45:50,259 INFO sqlalchemy.engine.Engine SELECT user.user_id AS user_user_id, user.name AS user_name, user.email AS user_email 
FROM user 
WHERE user.name = ?
 LIMIT ? OFFSET ?
2025-09-19 17:45:50,261 INFO sqlalchemy.engine.Engine [cached since 210.7s ago] ('Kavin', 1, 0)


In [24]:
all_record = session.query(User).all()

2025-09-19 17:47:01,719 INFO sqlalchemy.engine.Engine SELECT user.user_id AS user_user_id, user.name AS user_name, user.email AS user_email 
FROM user
2025-09-19 17:47:01,722 INFO sqlalchemy.engine.Engine [cached since 475.5s ago] ()


In [26]:
new_blog = Blog(title="How to successfully lose in life!", description="Buy this one and dig a 6 foot grave and jump into it!", author_id=kavin.user_id)
session.add(new_blog)
session.commit()

2025-09-19 17:47:18,879 INFO sqlalchemy.engine.Engine INSERT INTO blog (title, description, author_id) VALUES (?, ?, ?)
2025-09-19 17:47:18,880 INFO sqlalchemy.engine.Engine [cached since 489.1s ago] ('How to successfully lose in life!', 'Buy this one and dig a 6 foot grave and jump into it!', 1)
2025-09-19 17:47:18,887 INFO sqlalchemy.engine.Engine COMMIT


In [20]:
session.query(Blog).first().author.name

2025-09-19 17:42:28,260 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-19 17:42:28,262 INFO sqlalchemy.engine.Engine SELECT blog.blog_id AS blog_blog_id, blog.title AS blog_title, blog.description AS blog_description, blog.author_id AS blog_author_id 
FROM blog
 LIMIT ? OFFSET ?
2025-09-19 17:42:28,263 INFO sqlalchemy.engine.Engine [cached since 155.2s ago] (1, 0)
2025-09-19 17:42:28,266 INFO sqlalchemy.engine.Engine SELECT user.user_id AS user_user_id, user.name AS user_name, user.email AS user_email 
FROM user 
WHERE user.user_id = ?
2025-09-19 17:42:28,268 INFO sqlalchemy.engine.Engine [cached since 145.7s ago] (1,)


'Kavin'

### Deleting and Updating Record

In [27]:
session.query(User).filter(User.name=="Rashwanth").delete()
session.commit()

2025-09-19 17:48:18,996 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-19 17:48:18,999 INFO sqlalchemy.engine.Engine DELETE FROM user WHERE user.name = ?
2025-09-19 17:48:19,000 INFO sqlalchemy.engine.Engine [generated in 0.00133s] ('Rashwanth',)
2025-09-19 17:48:19,007 INFO sqlalchemy.engine.Engine COMMIT


In [29]:
session.query(User).filter(User.name=="Kavin").update({"email":"kavin@gmail.com"})
session.commit()

2025-09-19 17:49:39,288 INFO sqlalchemy.engine.Engine UPDATE user SET email=? WHERE user.name = ?
2025-09-19 17:49:39,289 INFO sqlalchemy.engine.Engine [cached since 16.54s ago] ('kavin@gmail.com', 'Kavin')
2025-09-19 17:49:39,292 INFO sqlalchemy.engine.Engine COMMIT


In [ ]:
### Joints

In [36]:
session.query(User.name, func.count(Blog.blog_id).label("Total No Blogs")).join(User.blogs).group_by(User.user_id).all()

2025-09-19 17:56:32,197 INFO sqlalchemy.engine.Engine SELECT user.name AS user_name, count(blog.blog_id) AS "Total No Blogs" 
FROM user JOIN blog ON user.user_id = blog.author_id GROUP BY user.user_id
2025-09-19 17:56:32,198 INFO sqlalchemy.engine.Engine [generated in 0.00117s] ()


[('Kavin', 3)]